In [245]:
# imports
import pandas as pd
import numpy as np
pd.options.plotting.backend = 'plotly'

from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


## read data

In [239]:
df_store = pd.read_pickle('data/df_daily.pkl')


## define functions

In [243]:
def preprocessing(ts):
    '''
    '''
    ts = ts['sales']/1e6
    ts.index.freq = 'D'
    return {'y':ts}

def cross_validation_result(data, model, model_name, horizon, rolls=4):
    '''
    '''
    mae_CVs = []
    rmse_CVs = []
    mape_CVs = []
    mase_CVs = []
    for i in range(rolls):
        y_train = data['y'].iloc[:-(rolls-i)*horizon]
        y_test = data['y'].iloc[np.r_[-(rolls-i)*horizon:-(rolls-i-1)*horizon]]
        
        model.fit(y=y_train)
        y_hat = model.predict(fh=list(range(1,horizon+1)))
        
        mae_CVs.append(round(mean_absolute_error(y_test, y_hat), 3))
        rmse_CVs.append(round(mean_squared_error(y_test, y_hat, square_root=True), 3))
        mape_CVs.append(round(mean_absolute_percentage_error(y_test, y_hat), 3))
        mase_CVs.append(round(mean_absolute_scaled_error(y_test, y_hat, y_train=y_train, sp=7), 3))
    
    return {'store':model_name,
           'sales': y_test,
           'mae_SNAIVE':np.mean(mae_CVs),
           'rmse_SNAIVE':np.mean(rmse_CVs),
           'mape_SNAIVE':np.mean(mape_CVs),
           'mase_SNAIVE':np.mean(mase_CVs),
           'fc_SNAIVE':y_hat,
           }


## fit on store data

In [246]:
horizon = 7
all_stores_result_CV = []
for store in df_store["store_id"].unique():
    model_name = "store_" + str(store)

    # data
    df_data = df_store[df_store["store_id"] == store].set_index("date")[["sales"]]
    df_data = preprocessing(df_data)

    # fit model to each store
    model = NaiveForecaster(sp=7, strategy="mean") # best strategy
    cv_score = cross_validation_result(data=df_data, model=model, model_name=model_name, horizon=horizon)

    # result
    all_stores_result_CV.append(cv_score)
all_stores_result_CV = pd.DataFrame(all_stores_result_CV)


## result

In [247]:
all_stores_result_CV.to_pickle('results/f8/SNAIVE_7.pkl')
all_stores_result_CV.head()


,store,sales,mae_SNAIVE,rmse_SNAIVE,mape_SNAIVE,mase_SNAIVE,fc_SNAIVE
0,store_307222,date 2021-01-25 24.848540 2021-01-26 18....,7.82875,10.07600,0.2515,0.50900,2021-01-25 17.433143 2021-01-26 19.11446...
1,store_307244,date 2021-01-25 17.889600 2021-01-26 18....,5.05825,6.30275,0.2595,0.39525,2021-01-25 14.368380 2021-01-26 14.75122...
2,store_307248,date 2021-01-25 21.487100 2021-01-26 23....,7.88775,9.26900,0.3830,0.75725,2021-01-25 12.218554 2021-01-26 12.39495...
3,store_320264,date 2021-01-25 12.109300 2021-01-26 11....,5.37550,7.82425,0.3055,0.53900,2021-01-25 10.469960 2021-01-26 10.82101...
4,store_328165,date 2021-01-25 41.708122 2021-01-26 5...,32.49675,45.40950,0.4430,1.32750,2021-01-25 28.437616 2021-01-26 28.82474...


# horizon = 14 days

In [248]:
horizon = 14
all_stores_result_CV = []
for store in df_store["store_id"].unique():
    model_name = "store_" + str(store)

    # data
    df_data = df_store[df_store["store_id"] == store].set_index("date")[["sales"]]
    df_data = preprocessing(df_data)

    # fit model to each store
    model = NaiveForecaster(sp=7, strategy="mean") # best strategy
    cv_score = cross_validation_result(data=df_data, model=model, model_name=model_name, horizon=horizon)

    # result
    all_stores_result_CV.append(cv_score)
all_stores_result_CV = pd.DataFrame(all_stores_result_CV)


## result

In [249]:
all_stores_result_CV.to_pickle('results/f8/SNAIVE_14.pkl')
all_stores_result_CV.head()


,store,sales,mae_SNAIVE,rmse_SNAIVE,mape_SNAIVE,mase_SNAIVE,fc_SNAIVE
0,store_307222,date 2021-01-18 18.799600 2021-01-19 27....,7.81800,10.00625,0.28850,0.50525,2021-01-18 17.425552 2021-01-19 19.06630...
1,store_307244,date 2021-01-18 10.078400 2021-01-19 17....,5.44350,6.82700,0.29500,0.42325,2021-01-18 14.392346 2021-01-19 14.73750...
2,store_307248,date 2021-01-18 22.384000 2021-01-19 19....,6.54350,8.07975,0.35450,0.62500,2021-01-18 12.161445 2021-01-19 12.35284...
3,store_320264,date 2021-01-18 10.991000 2021-01-19 11....,5.47900,7.19700,0.37675,0.54575,2021-01-18 10.467017 2021-01-19 10.81610...
4,store_328165,date 2021-01-18 38.661600 2021-01-19 4...,28.88275,41.19150,0.42725,1.18450,2021-01-18 28.380499 2021-01-19 28.73321...
